In [1]:
import os
import numpy as np
import parmed as pmd

from openmm import * 
from openmm.app import * 
from openmm.unit import * 

from sys import stdout

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [3]:
pdb = PDBFile('1ubq.pdb')

In [4]:
# forcefield = ForceField('amoeba2018.xml')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

In [5]:
modeller = Modeller(pdb.topology, pdb.positions)

In [6]:
_ = modeller.addHydrogens(forcefield)

In [7]:
modeller.addSolvent(forcefield, boxSize=Vec3(6, 6, 6)*nanometers, ionicStrength=0.1*molar, positiveIon='K+')

In [8]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME)
integrator = VerletIntegrator(0.001*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)
print('Saving...')
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('1ubq_sol.pdb', 'w'))
print('Done')

Saving...
Done


In [9]:
top = pmd.openmm.load_topology(modeller.topology, system, modeller.positions)

In [11]:
top.save('amber.prmtop', overwrite=1)

AttributeError: 'NoneType' object has no attribute 'used'